In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import json

In [2]:
train_data = []
train_inp_file = 'processed_data/GCDC/Clinton_train.jsonl'
with open(train_inp_file, 'r') as f:
    for line in f:
        json_obj = json.loads(line)
        # put <EOS> at the end of each sentence and add each sentence to the list
        for i in range(len(json_obj['sentences'])):
            json_obj['sentences'][i].append('<EOS>')
        # merge all sentences into one
        document = " ".join([word for sentence in json_obj['sentences'] for word in sentence])
        document = document.lower()
        label = json_obj['label']
        train_data.append((document, label))
   

test_data = []
test_inp_file = 'processed_data/GCDC/Clinton_test.jsonl'
with open(test_inp_file, 'r') as f:
    for line in f:
        json_obj = json.loads(line)
        # put <EOS> at the end of each sentence and add each sentence to the list
        for i in range(len(json_obj['sentences'])):
            json_obj['sentences'][i].append('<EOS>')
        # merge all sentences into one
        document = " ".join([word for sentence in json_obj['sentences'] for word in sentence])
        document = document.lower()
        label = json_obj['label']
        test_data.append((document, label))
        
print(len(train_data))
print(train_data[0])
print(train_data[1])
print(train_data[2])
print(len(test_data))
print(test_data[0])

# create vocabulary
vocab = set()
for document, label in train_data:
    for word in document.split():
        vocab.add(word.lower())
vocab = list(vocab)
vocab.append('<PAD>')
vocab.append('<UNK>')
print(len(vocab))
print(vocab[:10])


800
('two options the us views the transitional national council as the sole / only legitimate interlocutor of the libyan people during this interim period , as libyans come together to plan their own future and a permanent , inclusive constitutional system that protects the rights of all libyans . <eos> this is in contrast to the qadhafi regime , which has lost all legitimacy to rule . <eos> the us views the transitional national council as the legitimate interlocutor of the libyan people during this interim period , as libyans come together to plan their own future and a permanent , inclusive constitutional system that protects the rights of all libyans . <eos> this is in contrast to the qadhafi regime , which has lost all legitimacy to rule . <eos> the inc is the institution through which we are engaging the libyan people at this time . <eos>', 3)
("ambassador , we just received an email from the adoption service provider about these cases . <eos> i am currently reviewing the files 

In [3]:
# transform the documents into list of indices
def transform_doc(document, vocab):
    indices = []
    for word in document.split():
        if word.lower() in vocab:
            indices.append(vocab.index(word.lower()))
        else:
            indices.append(vocab.index('<UNK>'))
    return indices

# do it for all the sentences
train_data = [(transform_doc(document, vocab), label) for document, label in train_data]
test_data = [(transform_doc(document, vocab), label) for document, label in test_data]
# print(train_data[0])
# print(test_data[0])

# get the max length of the documents and also average length
total_len = 0
max_len = 0
count = 0
for document, label in train_data:
    max_len = max(max_len, len(document))
    total_len += len(document)
    count += 1

print(f'max doc length : {max_len}')
print(f'average doc length : {total_len/count}')

# pad the sentences to make them of same length
def pad_doc(document, max_len):
    if len(document) < max_len:
        document += [vocab.index('<PAD>')] * (max_len - len(document))
    return document

train_data = [(pad_doc(document, max_len), label) for document, label in train_data]
test_data = [(pad_doc(document, max_len), label) for document, label in test_data]
# print(train_data[0])
# print(len(train_data[0][0]))
# print(test_data[0])

# batchify the data after converting to tensors
class Batchify(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx][0]), torch.tensor(self.data[idx][1])
    
train_data = Batchify(train_data)
test_data = Batchify(test_data)

# create dataloaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)


max doc length : 415
average doc length : 194.9475


In [4]:
# define a transformer model for document coherence

class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout, max_len):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.transformer = nn.Transformer(embedding_dim, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout)
        self.linear = nn.Linear(embedding_dim, 2)
        self.max_len = max_len
    def forward(self, src):
        src = self.embedding(src)
        src = src.transpose(0, 1)
        src = self.transformer(src, src)
        src = src.transpose(0, 1)
        src = self.linear(src)
        return src
    
model = TransformerModel(len(vocab), 128, 4, 2, 2, 256, 0.2, 100)
# print(model)

In [5]:
# define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# train the model
def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for i, (sentence, label) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(sentence)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print('Epoch: {}, Loss: {}'.format(epoch, total_loss / len(train_loader)))

# test the model
def test(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (sentence, label) in enumerate(test_loader):
            output = model(sentence)
            loss = criterion(output, label)
            total_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(label.view_as(pred)).sum().item()
    print('Loss: {}, Accuracy: {}'.format(total_loss / len(test_loader), correct / len(test_loader.dataset)))

train(model, train_loader, criterion, optimizer, 10)
test(model, test_loader, criterion)

# save the model
torch.save(model.state_dict(), 'transformer.pt')

RuntimeError: Expected target size [32, 2], got [32]